# MyoSDK Tutorial

## What is MyoSDK?

MyoSDK is a Python library that allows you to **Retarget motion** from marker datas onto a 3D character.

## What You'll Learn

This tutorial will walk you through the complete process:
1. Setting up the MyoSDK client
2. Apply marker data describing a motion to a 3D character (retargeting)
3. Downloading the final animated character file

## Prerequisites

Before starting, make sure you have:
- **Python 3.8 or higher** installed
- **An API key** from MyoLab (you'll need to sign up and get your API key)
- **A Mocap file in c3d format** showing a person moving
- **A Markerset file ** describing approximately where the markers are placed on the body
- **Basic Python knowledge** (understanding variables, functions, and basic syntax)

## Installation

If you haven't installed `myosdk` yet, run this command in your terminal:
```bash
pip install myosdk
```

In [ ]:
# Step 1: Import required libraries
import os
from myosdk import Client

# Step 2: Set up your API key
# The API key is like a password that authenticates you with the MyoSDK service.
# You can get your API key from your MyoLab account dashboard at dev.myolab.ai.
# Instructions are available at https://docs.myolab.ai/docs/myosdk/getting-started/api-key

# Option 1 (Recommended): Set it as an environment variable
# In your terminal, run: export MYOSDK_API_KEY="your-api-key-here"
# Or on Windows: set MYOSDK_API_KEY=your-api-key-here

# Option 2: Replace "..." below with your actual API key (less secure, but easier for testing)
api_key = os.getenv("MYOSDK_API_KEY", "...")
api_key = os.getenv("MYOSDK_API_KEY", "v2m_7FdkSkKXsVJvSaoS:Qu6pW0kkMOs8vqOAKKgp0bv0suQfNzzb")

# Step 3: Create the client
# The client is your connection to the MyoSDK service. It handles all communication
# with the servers. The base_url tells it which server to connect to.
client = Client(api_key=api_key, base_url="https://v2m-alb-us-east-1.myolab.ai")

# Verify the connection worked
print("Client initialized successfully.")
print("✓ You're ready to use MyoSDK!")

Client initialized successfully.


## 1. Upload Input Assets

We need to upload a c3d and markerset files to the platform. These will be used to...

In [ ]:
c3d_path = "../data/35_30.c3d"
markerset_path = "../markersets/cmu_markerset.xml"

# Upload C3D motion capture file
c3d_asset = client.assets.upload_file(c3d_path)
print(c3d_asset)
print("<<< add validation in sdk e.g. client.validate_mocap_data(c3d_path)")

# Upload markerset XML file
markerset_asset = client.assets.upload_file(markerset_path)
print(markerset_asset)
print("<<< add validation in sdk e.g. client.validate_markerset(markerset_path)")


{'asset_id': '019acc02-712a-78be-80ee-f4ec7844421e', 'verified': True, 'size_bytes': 142816, 'checksum_sha256': 'b32cea92fcfb28de92a628d1de61d13a5d40be1bed1485a4d4253582ce7fb772', 'message': 'Asset upload verified successfully'}
{'asset_id': '019acc02-727f-717b-96e1-817ce8241e4c', 'verified': True, 'size_bytes': 4806, 'checksum_sha256': '9d6e1ee597212f85af8165bedcf65e4108faa7b6e145232b9f31dd1bbf41d92b', 'message': 'Asset upload verified successfully'}


## 2. Motion Retargeting

Use the extracted motion data (trackers) to retarget onto a 3D character. This is called "retargeting."

**What is retargeting?** Think of it like this: you recorded someone dancing, extracted their dance moves, and now you're applying those exact same moves to a 3D character. The character will move in the same way as the person in your video.

**What you need:**
- The `c3d_asset` describing the motion data
- The `markerset_asset` describing the association of motion data to body parts


In [6]:
# Using a specific character ID (replace with one from the list above if needed)
character_id = "0199e278-06e6-7726-a6ba-9929f79005e8"
character_version = "v1.0.0"
## TODO: remove these info and add as default values in the sdk

# Start retargeting job
job = client.jobs.start_retarget(
    c3d_asset_id=c3d_asset["asset_id"],
    markerset_asset_id=markerset_asset["asset_id"],
    character_id=character_id,
    character_version=character_version,
)

# Wait for job to complete
result = client.jobs.wait(job["job_id"])

qpos_asset_id = result["output"]["qpos_asset_id"]
print(qpos_asset_id)

019acc0b-5b0a-7051-844c-b57d1bb7dfb0


## 3. Download Result

Finally, we'll download the generated animated character file. The file will be in GLB format, which is a 3D file format that can be used in many 3D software and game engines.

**What is a GLB file?** GLB is a binary format for 3D models that includes the character geometry, textures, and animation data all in one file. You can:
- View it in 3D viewers like Blender, Unity, Unreal Engine
- Use it in web applications with libraries like Three.js
- Import it into game engines or animation software

**Note:** Make sure the output directory exists, or the download will create it for you.

In [7]:
output_path = "tmp-assets/tutorial_output.npy"

print(f"Downloading result to {output_path}...")
client.assets.download(qpos_asset_id, output_path)
print("Download complete!")

Download complete!
